In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,23271
2,Huelva,Confirmados PDIA 14 días,4375
3,Huelva,Tasa PDIA 14 días,"852,5439912699495"
4,Huelva,Confirmados PDIA 7 días,1563
5,Huelva,Total Confirmados,23501
6,Huelva,Curados,12136
7,Huelva,Fallecidos,208


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  23271.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6154.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 422 personas en los últimos 7 días 

Un positivo PCR cada 138 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,23271.0,1563.0,4375.0,513170.0,304.577430,852.543991,570.0
Huelva-Costa,14130.0,928.0,2630.0,289548.0,320.499537,908.312266,347.0
Condado-Campiña,6443.0,486.0,1324.0,156231.0,311.077827,847.463052,189.0
Huelva (capital),6154.0,341.0,1041.0,143837.0,237.073910,723.735895,127.0
Isla Cristina,1916.0,122.0,335.0,21393.0,570.279998,1565.932782,54.0
Lepe,1872.0,138.0,359.0,27880.0,494.978479,1287.661406,52.0
Ayamonte,1135.0,115.0,305.0,21104.0,544.920394,1445.223654,48.0
Bollullos Par del Condado,546.0,68.0,184.0,14387.0,472.648919,1278.932370,30.0
Almonte,741.0,60.0,144.0,24507.0,244.828008,587.587220,28.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Berrocal,19.0,5.0,9.0,303.0,1650.165017,2970.297030,2.0
Villablanca,153.0,35.0,66.0,2885.0,1213.171577,2287.694974,18.0
Villalba del Alcor,436.0,26.0,74.0,3366.0,772.430184,2198.455140,14.0
Paymogo,70.0,10.0,25.0,1162.0,860.585198,2151.462995,3.0
San Bartolomé de la Torre,187.0,13.0,59.0,3761.0,345.652752,1568.731720,3.0
Isla Cristina,1916.0,122.0,335.0,21393.0,570.279998,1565.932782,54.0
Cabezas Rubias,28.0,2.0,11.0,706.0,283.286119,1558.073654,0.0
Ayamonte,1135.0,115.0,305.0,21104.0,544.920394,1445.223654,48.0
Palma del Condado (La),950.0,52.0,153.0,10801.0,481.436904,1416.535506,17.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cerro de Andévalo (El),131.0,1.0,15.0,2327.0,42.973786,644.606790,1.0,0.066667
Encinasola,116.0,1.0,7.0,1310.0,76.335878,534.351145,0.0,0.142857
Nerva,145.0,3.0,20.0,5169.0,58.038305,386.922035,0.0,0.150000
Alosno,118.0,3.0,17.0,3933.0,76.277651,432.240020,1.0,0.176471
Zalamea la Real,82.0,3.0,17.0,3054.0,98.231827,556.647020,0.0,0.176471
Cabezas Rubias,28.0,2.0,11.0,706.0,283.286119,1558.073654,0.0,0.181818
Punta Umbría,609.0,30.0,142.0,15355.0,195.376099,924.780202,11.0,0.211268
Aracena,451.0,10.0,47.0,8255.0,121.138704,569.351908,7.0,0.212766
San Bartolomé de la Torre,187.0,13.0,59.0,3761.0,345.652752,1568.731720,3.0,0.220339
